In [2]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [3]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [5]:
# train set을 한 번 더 나눔 (훈련 세트, 검증 세트)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [6]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [7]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [8]:
# 교차 검증 함수를 이용하여 교차 검증 수행. cross_validate 함수는 기본적으로 5-폴드 교차 검증을 수행한다.
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00949883, 0.00761867, 0.00787354, 0.00752044, 0.00706935]), 'score_time': array([0.00091267, 0.00085616, 0.00106692, 0.00072002, 0.00072145]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [9]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
# 교차 검증 시 훈련 세트를 섞으려면 분할기(splitter)를 지정해야 한다. cross_validate() 함수는 기본적으로 회귀 모델일 경우 KFold 모델, 분류 모델일 경우 StratifeidKFold 모델을 사용한다.
# 앞서 수행한 교차 검증은 다음 코드와 동일하다. (앞선 수행에서는 train_test_split으로 훈련 세트를 섞어주었기 때문.)
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [14]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [15]:
# 그리드 서치, GridSearchCV 클래스는 하이퍼파라미터 탐색과 교차검증을 한번에 수행함. 
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=-1) # cv 매개변수 기본값은 5(5-폴드 교차 검증 수행), 따라서 25개의 모델을 훈련한다. n_jobs 매개변수는 병렬 실행에 사용할 CPU 코어 수 지정함
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [16]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [17]:
# 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장되어 있다.
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [18]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [19]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [21]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), 'max_depth': range(5, 20, 1), 'min_samples_split': range(2, 100, 10)}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=-1) # cv 매개변수 기본값은 5(5-폴드 교차 검증 수행), 따라서 25개의 모델을 훈련한다. n_jobs 매개변수는 병렬 실행에 사용할 CPU 코어 수 지정함
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [22]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [23]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [26]:
# 랜덤 서치에 이용할 scipy 라이브러리의 확률 분포 클래스 
from scipy.stats import uniform, randint
rgen = randint(0, 10)
rgen.rvs(10)

array([7, 7, 3, 0, 1, 0, 0, 4, 7, 8])

In [28]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([102,  93, 113, 100,  90, 115,  92, 109,  91,  95]))

In [29]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.80617979, 0.73538978, 0.89235149, 0.62051831, 0.89732168,
       0.32682878, 0.30276044, 0.22955755, 0.99487762, 0.28928484])

In [30]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [31]:
# 랜덤 서치
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f4409f18f10>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f4408d2f090>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f4408c84e50>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f4408b65210>},
                   random_state=42)

In [32]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [33]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [34]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
